In [1]:
from llmin.similarity import sim_matrix

from transformers import AutoModelForCausalLM
import torch

target_model = "mistralai/Mistral-7B-v0.1"

model = AutoModelForCausalLM.from_pretrained(
    target_model,
    device_map = "cuda",
    torch_dtype = torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from llmin.utils import get_linear_module_names
linear_module_names = get_linear_module_names(model)
linear_module_names

['model.layers.0.self_attn.q_proj',
 'model.layers.0.self_attn.k_proj',
 'model.layers.0.self_attn.v_proj',
 'model.layers.0.self_attn.o_proj',
 'model.layers.0.mlp.gate_proj',
 'model.layers.0.mlp.up_proj',
 'model.layers.0.mlp.down_proj',
 'model.layers.1.self_attn.q_proj',
 'model.layers.1.self_attn.k_proj',
 'model.layers.1.self_attn.v_proj',
 'model.layers.1.self_attn.o_proj',
 'model.layers.1.mlp.gate_proj',
 'model.layers.1.mlp.up_proj',
 'model.layers.1.mlp.down_proj',
 'model.layers.2.self_attn.q_proj',
 'model.layers.2.self_attn.k_proj',
 'model.layers.2.self_attn.v_proj',
 'model.layers.2.self_attn.o_proj',
 'model.layers.2.mlp.gate_proj',
 'model.layers.2.mlp.up_proj',
 'model.layers.2.mlp.down_proj',
 'model.layers.3.self_attn.q_proj',
 'model.layers.3.self_attn.k_proj',
 'model.layers.3.self_attn.v_proj',
 'model.layers.3.self_attn.o_proj',
 'model.layers.3.mlp.gate_proj',
 'model.layers.3.mlp.up_proj',
 'model.layers.3.mlp.down_proj',
 'model.layers.4.self_attn.q_proj',


In [7]:
from tqdm import tqdm

linear_module_names = get_linear_module_names(model)

def calculate_similarities(
    linear_module_name: str = "model.layers.{idx}.self_attn.q_proj.weight",
    num_layers = 32
):

    similarities = []
    for i in tqdm(range(0, num_layers)):
        for j in range(0, num_layers):
            if i != j:
                sim = sim_matrix(
                    a = model.state_dict()[linear_module_name.format(idx = i)],
                    b = model.state_dict()[linear_module_name.format(idx = j)]
                ).to("cpu")
                idx = (sim==torch.max(sim)).nonzero()[0].to("cpu")
                similarities.append({
                    "sim": sim[idx[0].item(), idx[1].item()],
                    "layer_1": linear_module_name.format(idx = i),
                    "layer_2": linear_module_name.format(idx = j)
                })
    return similarities

similarities = calculate_similarities()

100%|██████████| 32/32 [00:23<00:00,  1.38it/s]


In [13]:
sorted(similarities, key = lambda item: item["sim"], reverse = True)

[{'sim': tensor(0.8633, dtype=torch.bfloat16),
  'layer_1': 'model.layers.21.self_attn.q_proj.weight',
  'layer_2': 'model.layers.22.self_attn.q_proj.weight'},
 {'sim': tensor(0.8633, dtype=torch.bfloat16),
  'layer_1': 'model.layers.22.self_attn.q_proj.weight',
  'layer_2': 'model.layers.21.self_attn.q_proj.weight'},
 {'sim': tensor(0.8359, dtype=torch.bfloat16),
  'layer_1': 'model.layers.22.self_attn.q_proj.weight',
  'layer_2': 'model.layers.23.self_attn.q_proj.weight'},
 {'sim': tensor(0.8359, dtype=torch.bfloat16),
  'layer_1': 'model.layers.23.self_attn.q_proj.weight',
  'layer_2': 'model.layers.22.self_attn.q_proj.weight'},
 {'sim': tensor(0.8242, dtype=torch.bfloat16),
  'layer_1': 'model.layers.24.self_attn.q_proj.weight',
  'layer_2': 'model.layers.25.self_attn.q_proj.weight'},
 {'sim': tensor(0.8242, dtype=torch.bfloat16),
  'layer_1': 'model.layers.25.self_attn.q_proj.weight',
  'layer_2': 'model.layers.24.self_attn.q_proj.weight'},
 {'sim': tensor(0.8164, dtype=torch.bflo

In [7]:
!pip install tqdm -q

  0%|          | 0/225 [00:00<?, ?it/s]


RuntimeError: a Tensor with 4194304 elements cannot be converted to Scalar

In [5]:
similarities

[{'sim': tensor(0.3223, dtype=torch.bfloat16),
  'idx_0': tensor(2545),
  'idx_1': tensor(2024)},
 {'sim': tensor(0.1670, dtype=torch.bfloat16),
  'idx_0': tensor(3006),
  'idx_1': tensor(1463)},
 {'sim': tensor(0.1738, dtype=torch.bfloat16),
  'idx_0': tensor(1749),
  'idx_1': tensor(2303)},
 {'sim': tensor(0.1260, dtype=torch.bfloat16),
  'idx_0': tensor(2805),
  'idx_1': tensor(2678)},
 {'sim': tensor(0.1040, dtype=torch.bfloat16),
  'idx_0': tensor(49),
  'idx_1': tensor(1078)},
 {'sim': tensor(0.0996, dtype=torch.bfloat16),
  'idx_0': tensor(1813),
  'idx_1': tensor(3255)},
 {'sim': tensor(0.1050, dtype=torch.bfloat16),
  'idx_0': tensor(3189),
  'idx_1': tensor(122)},
 {'sim': tensor(0.0957, dtype=torch.bfloat16),
  'idx_0': tensor(1634),
  'idx_1': tensor(3275)},
 {'sim': tensor(0.0889, dtype=torch.bfloat16),
  'idx_0': tensor(1062),
  'idx_1': tensor(1270)},
 {'sim': tensor(0.0908, dtype=torch.bfloat16),
  'idx_0': tensor(3189),
  'idx_1': tensor(1787)},
 {'sim': tensor(0.1030,

In [ ]:
from torchmetrics.functional import pairwise_cosine_similarity

pairwise_cosine_similarity(
    model.state_dict()["model.layers.0.self_attn.q_proj.weight"],
    model.state_dict()["model.layers.1.self_attn.q_proj.weight"]
)

In [ ]:
from torch import nn
import torch

cos = nn.CosineSimilarity(dim = 0, eps=1e-6)
output = cos(
    model.state_dict()["model.layers.0.self_attn.q_proj.weight"],
    model.state_dict()["model.layers.1.self_attn.q_proj.weight"]
)
output[torch.argmax(output)]

In [ ]:
model.state_dict()["model.layers.0.self_attn.q_proj.weight"][0].shape

In [ ]:
from torchmetrics import CosineSimilarity

cosine_similarity = CosineSimilarity(reduction = 'mean')
cosine_similarity(
    model.state_dict()["model.layers.0.self_attn.q_proj.weight"],
    model.state_dict()["model.layers.1.self_attn.q_proj.weight"]
)

In [ ]:
cosine_similarity(
    model.state_dict()["model.layers.0.self_attn.q_proj.weight"],
    model.state_dict()["model.layers.7.self_attn.q_proj.weight"]
)